In [28]:
pip install chromedriver-autoinstaller

Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [30]:
import chromedriver_autoinstaller
from selenium import webdriver
import time

class Driver:
    @staticmethod
    def driver_init():
        chromedriver_autoinstaller.install()
        options = webdriver.ChromeOptions()
#         options.add_argument('headless') # jupyter notebook 환경에서 주석처리
        options.add_argument('lang=ko_KR')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-notifications')
        options.add_argument('--disable-popup-blocking')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-browser-side-navigation')
        options.add_argument('disable-gpu')

        driver = webdriver.Chrome(options=options)
        driver.set_window_size(1600, 1100)

        return driver

In [31]:
driver = Driver.driver_init()

In [32]:
def get_paths(gender, url, men, women):
    product_paths = []
    category_str = ''
    page_list = []
    
    if gender == 1:
        category_str = '&category=00010001000'
        page_list = men
    elif gender == 2:
        category_str = '&category=00020001000'
        page_list = women

    ## 카테고리 분류 루프
    for category_cnt in range(len(page_list)):
        pages_cnt = page_list[category_cnt][0]
        cat = page_list[category_cnt][1]
        ## 각 옷 분류의 페이지 갯수 루프
        for page_num in range(pages_cnt):
            try:
                page = url + "?page=" + str(page_num+1) + category_str + str(category_cnt+1)
                driver.get(page)
                time.sleep(1)
            except:
                continue

            products = driver.find_elements('xpath', '//div[@id="searchedItemDisplay"]/ul/li')
            products_cnt = len(products)

            ## 각 product의 path를 product_paths 리스트에 저장
            for i in range(products_cnt):
                product = driver.find_elements('xpath', '//div[@class="searched_item_display"]/ul/li')[i]
                product_path = product.find_element('xpath', './div/div/a').get_attribute('href')
                product_paths.append((product_path, cat, gender))

    print(len(product_paths))
    
    return product_paths
    
        
        

In [33]:
def crawl_pages(product_paths):
    
    base_dict= {"baseCat":[], "baseName": [], "baseCode": [], "baseUrl": [], "colorName": [],
            "baseGender": [], "price": [], "basePrice": [], "colorFabric": [], "imageUrl": [],
            "size": [], "isAvailable": []
            }
    
    ## 드라이버 초기화
    driver = Driver.driver_init()
    ## 각 product에 대한 정보 추출
    for product_path, cat, gender in product_paths:
        try:
            driver.get(product_path)
        except:
            continue
        
        imageURL = []
        imageURL_list = driver.find_elements('xpath', '//div[@class="slides_container hide"]/div[@class="viewImgWrap"]/img')
        for image_element in imageURL_list:
            imageURL.append(image_element.get_attribute("src"))
            
        color_fabric = driver.find_element('xpath','//div[@id="materials"]').get_attribute("textContent")

        
        baseURL = product_path
        base_gender = gender
        text_info = driver.find_element('xpath', '//div[@class="goods_thumbs_spec"]')

        base_name = text_info.find_element('xpath', '//div[@class="detail_title_area"]/h3').get_attribute("textContent")
        base_code = base_name
        
        try:
            if len(text_info.find_elements('xpath', '//p[@class="sale_price sale_txt"]')) == 0:
                price = text_info.find_element('xpath', '//p[@class="sale_price"]/span[@class="num"]').get_attribute("textContent")
                base_price = price
            else:
                price = text_info.find_element('xpath', '//p[@class="sale_price sale_txt"]/span[@class="num"]').get_attribute("textContent")
                base_price = text_info.find_element('xpath', '//p[@class="org_price"]/span[@class="dst_th_size"]/s/span[@class="num"]').get_attribute("textContent")
        except:
            continue
            
        colors_list = text_info.find_elements('xpath','//ul[@class="optionBox color"]/li')
        
        for color_element in colors_list:
            color_name = color_element.get_attribute("data-value")
            try:
                color_element.click()
            except:
                continue
            
            size_list = text_info.find_elements('xpath','//ul[@class="optionBox size"]/li')
            for size_element in size_list:
                size = size_element.get_attribute("data-value")
                try:
                    size_element.click()
                except:
                    continue
                if size_element.get_attribute("class") == "soldout":
                    is_available = False
                else:
                    is_available = True
                    
            
                base_dict["baseCat"].append(cat)
                base_dict["baseName"].append(base_name)
                base_dict["baseCode"].append(base_code)
                base_dict["baseUrl"].append(baseURL)
                base_dict["colorName"].append(color_name)
                base_dict["baseGender"].append(base_gender)
                base_dict["price"].append(price)
                base_dict["basePrice"].append(base_price)
                base_dict["colorFabric"].append(color_fabric)
                base_dict["imageUrl"].append(imageURL)
                base_dict["size"].append(size)
                base_dict["isAvailable"].append(is_available)
    return base_dict

In [34]:
men = [(2,'top'), (1,'top'), (2,'bottom'), (1,'top'), (1,'top'), (2,'top'), (1,'bottom')]
women = [(2, 'top'), (1, 'top'), (1, 'bottom'), (1, 'top'), (1, 'top'), (2, 'top'), (1, 'other'), (1, 'bottom')]

url = 'https://arcteryx.co.kr/goods/catalog'

product_paths = get_paths(1, url, men, women) + get_paths(2, url, men, women)
print(len(product_paths))

248
215
463


In [35]:
import pandas as pd

In [36]:
dict1 = crawl_pages(product_paths[:200])
df1 = pd.DataFrame.from_dict(dict1)

print(df1)


     baseCat            baseName            baseCode  \
0        top  [FW23] 알파 SV 재킷 남성  [FW23] 알파 SV 재킷 남성   
1        top  [FW23] 알파 SV 재킷 남성  [FW23] 알파 SV 재킷 남성   
2        top  [FW23] 알파 SV 재킷 남성  [FW23] 알파 SV 재킷 남성   
3        top  [FW23] 알파 SV 재킷 남성  [FW23] 알파 SV 재킷 남성   
4        top  [FW23] 알파 SV 재킷 남성  [FW23] 알파 SV 재킷 남성   
...      ...                 ...                 ...   
3719     top     [SS23] 코막 후디 남성     [SS23] 코막 후디 남성   
3720     top     [SS23] 코막 후디 남성     [SS23] 코막 후디 남성   
3721     top     [SS23] 코막 후디 남성     [SS23] 코막 후디 남성   
3722     top     [SS23] 코막 후디 남성     [SS23] 코막 후디 남성   
3723     top     [SS23] 코막 후디 남성     [SS23] 코막 후디 남성   

                                          baseUrl       colorName  baseGender  \
0     https://arcteryx.co.kr/goods/view?no=683538      BLUE TETRA           1   
1     https://arcteryx.co.kr/goods/view?no=683538      BLUE TETRA           1   
2     https://arcteryx.co.kr/goods/view?no=683538      BLUE TETRA           1   
3  

In [37]:
dict2 = crawl_pages(product_paths[200:])
df2 = pd.DataFrame.from_dict(dict2)

print(df2)


     baseCat                     baseName                     baseCode  \
0        top  [SS23] 스카이라인 SS 셔츠 남성 - 멜란지  [SS23] 스카이라인 SS 셔츠 남성 - 멜란지   
1        top  [SS23] 스카이라인 SS 셔츠 남성 - 멜란지  [SS23] 스카이라인 SS 셔츠 남성 - 멜란지   
2        top  [SS23] 스카이라인 SS 셔츠 남성 - 멜란지  [SS23] 스카이라인 SS 셔츠 남성 - 멜란지   
3        top  [SS23] 스카이라인 SS 셔츠 남성 - 멜란지  [SS23] 스카이라인 SS 셔츠 남성 - 멜란지   
4        top  [SS23] 스카이라인 SS 셔츠 남성 - 멜란지  [SS23] 스카이라인 SS 셔츠 남성 - 멜란지   
...      ...                          ...                          ...   
3438  bottom             [SS20] 카일라 롬퍼 여성             [SS20] 카일라 롬퍼 여성   
3439  bottom             [SS20] 카일라 롬퍼 여성             [SS20] 카일라 롬퍼 여성   
3440  bottom             [SS20] 카일라 롬퍼 여성             [SS20] 카일라 롬퍼 여성   
3441  bottom             [SS20] 카일라 롬퍼 여성             [SS20] 카일라 롬퍼 여성   
3442  bottom             [SS20] 카일라 롬퍼 여성             [SS20] 카일라 롬퍼 여성   

                                          baseUrl       colorName  baseGender  \
0     https://arcteryx.co.kr/g

In [38]:
df_final = pd.concat([df1, df2])

In [39]:
df_final.to_csv('result.csv',index=False)